<a href="https://colab.research.google.com/github/boradj/machine-learning/blob/main/food-vision/food_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Important stuff

In [ ]:
##Mixed precision

## Check GPU

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-e01462c7-27a6-e341-fe5e-a8a29980984f)


##helper function

In [ ]:
# download it

In [ ]:
## use tensorflow datasets
import tensorflow_datasets as tfds


In [ ]:
datasets_list = tfds.list_builders()
print("food101" in datasets_list)

True


In [ ]:
(train_data, test_data), ds_info = tfds.load(name = "food101",
                                    split=["train", "validation"],
                                    shuffle_files=True,
                                    as_supervised = True,
                                    with_info= True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

KeyboardInterrupt: ignored

In [ ]:
# Features of 101
ds_info.features


In [ ]:
class_names = ds_info.features["label"].names


In [ ]:
train_data
train_one_sample = train_data.take(1)

In [ ]:
train_one_sample

In [ ]:
for image, label in train_one_sample:
  print(f"""
        Image shape:{image.shape},
        Image datatype:{image.dtype},
        Target class from Food101(tensor form):{label}
        class_name (str form):{class_name[label.numpy()]}""")

In [ ]:
image

In [ ]:
import tensorflow as tf
tf.reduce_min(image), tf.reduce_max(image)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image)
plt.title(class_names[label.numpy()])
plt.axis(False);